In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
import surprise

from recommenders.utils.general_utils import get_number_processors
from recommenders.utils.gpu_utils import get_cuda_version, get_cudnn_version
from recommenders.datasets.python_splitters import python_stratified_split

from benchmark_utils import * 

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Surprise version: {}".format(surprise.__version__))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))
n_cores = get_number_processors()
print("Number of cores: {}".format(n_cores))

%load_ext autoreload
%autoreload 2

System version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Pandas version: 1.3.4
Surprise version: 1.1.1
PyTorch version: 1.10.1+cu102
Cornac version: 1.14.1
Tensorflow version: 2.7.0
CUDA version: No CUDA in this machine
CuDNN version: No CUDNN in this machine
Number of cores: 48


In [2]:
# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
environments = {
    "svd": "python_cpu",
    "ncf": "python_gpu",
    "bpr": "python_cpu",
    "bivae": "python_gpu",
}

metrics = {
    "svd": ["rating", "ranking"],
    "ncf": ["ranking"],
    "bpr": ["ranking"],
    "bivae": ["ranking"],
}

In [4]:
svd_params = {
    "n_factors": 150,
    "n_epochs": 15,
    "lr_all": 0.005,
    "reg_all": 0.02,
    "random_state": SEED,
    "verbose": False
}

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 15,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

bpr_params = {
    "k": 200,
    "max_iter": 200,
    "learning_rate": 0.01,
    "lambda_reg": 1e-3,
    "seed": SEED,
    "verbose": False
}

bivae_params = {
    "k": 100,
    "encoder_structure": [200],
    "act_fn": "tanh",
    "likelihood": "pois",
    "n_epochs": 500,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "seed": SEED,
    "use_gpu": True,
    "verbose": False
}

params = {
    "svd": svd_params,
    "ncf": ncf_params,
    "bpr": bpr_params,
    "bivae": bivae_params,
}

In [5]:
prepare_training_data = {
    "svd": prepare_training_svd,
    "ncf": prepare_training_ncf,
    "bpr": prepare_training_cornac,
    "bivae": prepare_training_cornac,
}

In [6]:
prepare_metrics_data = {
    "als": lambda train, test: prepare_metrics_als(train, test),
    "fastai": lambda train, test: prepare_metrics_fastai(train, test),    
}

In [7]:
trainer = {
    "svd": lambda params, data: train_svd(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "bpr": lambda params, data: train_bpr(params, data),
    "bivae": lambda params, data: train_bivae(params, data),
}

In [8]:
ranking_predictor = {
    "svd": lambda model, test, train: recommend_k_svd(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "bpr": lambda model, test, train: recommend_k_cornac(model, test, train),
    "bivae": lambda model, test, train: recommend_k_cornac(model, test, train),
}

ranking_evaluator = {
    "svd": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bpr": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bivae": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}

def generate_summary(data, algo, k, train_time, time_ranking, ranking_metrics):
    summary = {"Data": data, "Algo": algo, "K": k, "Train time (s)": train_time, "Recommending time (s)": time_ranking}

    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(ranking_metrics)
    return summary

In [9]:
algorithms = ["svd", "ncf", "bpr", "bivae"]

In [11]:
# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

# Load the dataset
df = pd.read_csv('data/ml-1m/ratings.dat', sep='::', header=None, engine='python')
df.columns = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
df[DEFAULT_RATING_COL] = df[DEFAULT_RATING_COL].astype(np.float64)
print("Size of Movielens {}: {}".format("1m", df.shape))

# Split the dataset
df_train, df_test = python_stratified_split(df,
                                            ratio=0.75, 
                                            min_rating=1, 
                                            filter_by="item", 
                                            col_user=DEFAULT_USER_COL, 
                                            col_item=DEFAULT_ITEM_COL
                                            )

# Loop through the algos
for algo in algorithms:
      
    # Data prep for training set
    train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
    
    # Get model parameters
    model_params = params[algo]
      
    # Train the model
    model, time_train = trainer[algo](model_params, train)
            
    # Predict and evaluate
    train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
    
    if "ranking" in metrics[algo]:
        # Predict for ranking
        top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
        
        # Evaluate for rating
        for k in [10, 50, 100]:
            
            DEFAULT_K = k
            rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
            
            # Record results
            summary = generate_summary('MovieLens(1M)', algo, DEFAULT_K, time_train, time_ranking, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
            
    print("algo: {0}, training time: {1}, ranking time: {2}".format(algo, time_train, time_ranking))
    
print("\nComputation finished")

Size of Movielens 1m: (1000209, 4)
algo: svd, training time: 59.8584, ranking time: 229.4867


/dev/shm/mert.burabak/.conda/envs/mert_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


algo: ncf, training time: 770.3555, ranking time: 61.5513
algo: bpr, training time: 93.3865, ranking time: 28.6172
algo: bivae, training time: 244.9366, ranking time: 33.6664

Computation finished


In [13]:
df_results

,Data,Algo,K,Train time (s),Recommending time (s),MAP,nDCG@k,Precision@k,Recall@k
1,MovieLens(1M),svd,10,59.8584,229.4867,0.008828,0.089320,0.082856,0.021582
2,MovieLens(1M),svd,50,59.8584,229.4867,0.015470,0.085347,0.057870,0.078325
3,MovieLens(1M),svd,100,59.8584,229.4867,0.019179,0.100031,0.048223,0.130349
4,MovieLens(1M),ncf,10,770.3555,61.5513,0.065005,0.353338,0.321617,0.110738
5,MovieLens(1M),ncf,50,770.3555,61.5513,0.121895,0.340117,0.208269,0.317386
6,MovieLens(1M),ncf,100,770.3555,61.5513,0.148802,0.379354,0.159538,0.460706
7,MovieLens(1M),bpr,10,93.3865,28.6172,0.083546,0.390911,0.357777,0.142510
8,MovieLens(1M),bpr,50,93.3865,28.6172,0.157444,0.394178,0.229723,0.388976
9,MovieLens(1M),bpr,100,93.3865,28.6172,0.187950,0.438218,0.171663,0.537402
10,MovieLens(1M),bivae,10,244.9366,33.6664,0.093725,0.433974,0.396621,0.151100
